In [219]:
import torch
import numpy as np

# Training data

In [220]:
# input (temp,rainfall,humidy)
inputs = np.array([[73,67,43],[91,88,64],[87,134,58],[102,43,37],[69,96,70]],dtype='float32')

In [221]:
# Targets (apple,oranges)
targets = np.array([[56,70],[81,101],[119,133],[22,37],[103,119]],dtype='float32')

In [222]:
# convert into tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Linear regression from scratch

In [223]:
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[0.8041, 0.7573, 0.1959],
        [0.1137, 1.0792, 2.0290]], requires_grad=True)
tensor([-0.2315,  0.8012], requires_grad=True)


In [224]:
# example of one iteration
inputs @ w.t() + b

tensor([[117.6269, 168.6523],
        [152.1172, 235.9709],
        [182.5602, 272.9844],
        [121.5941, 133.8739],
        [141.6619, 254.2781]], grad_fn=<AddBackward0>)

In [225]:
# creating the model
def model(x):
    return x @ w.t() + b

In [226]:
pred = model(inputs)
pred

tensor([[117.6269, 168.6523],
        [152.1172, 235.9709],
        [182.5602, 272.9844],
        [121.5941, 133.8739],
        [141.6619, 254.2781]], grad_fn=<AddBackward0>)

In [227]:
# pretty bad model 
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Improve model

In [228]:
# calculating the error
diff = pred - targets
torch.sum(diff * diff) / diff.numel()

tensor(9953.8916, grad_fn=<DivBackward0>)

# Loss Function

In [229]:
# MSE loss

def mse (t1,t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [230]:
loss = mse(pred,targets)
print(loss)

tensor(9953.8916, grad_fn=<DivBackward0>)


# Compute gradients

In [231]:
loss.backward()

In [232]:
print(w)
print(w.grad)

tensor([[0.8041, 0.7573, 0.1959],
        [0.1137, 1.0792, 2.0290]], requires_grad=True)
tensor([[ 5865.2886,  5379.6968,  3455.8540],
        [10175.5859, 10879.4639,  6810.6152]])


In [233]:
print(b)
print(b.grad)

tensor([-0.2315,  0.8012], requires_grad=True)
tensor([ 66.9121, 121.1519])


# Adjust weights and biases

In [234]:
print(w)
print(w.grad)

tensor([[0.8041, 0.7573, 0.1959],
        [0.1137, 1.0792, 2.0290]], requires_grad=True)
tensor([[ 5865.2886,  5379.6968,  3455.8540],
        [10175.5859, 10879.4639,  6810.6152]])


In [235]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [236]:
w, b

(tensor([[0.7454, 0.7035, 0.1614],
         [0.0119, 0.9704, 1.9609]], requires_grad=True),
 tensor([-0.2322,  0.7999], requires_grad=True))

In [237]:
preds = model(inputs)
loss = mse(preds,targets)
print(loss)

tensor(6825.2671, grad_fn=<DivBackward0>)


In [238]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# Train model using gradient descent

In [239]:
# making predictions
preds = model(inputs)
print(preds)

tensor([[108.2542, 151.0051],
        [139.8333, 212.7772],
        [168.2436, 245.6018],
        [112.0189, 116.2954],
        [130.0306, 232.0440]], grad_fn=<AddBackward0>)


In [240]:
# calculate the loss
loss = mse(preds,targets)
print(loss)

tensor(6825.2671, grad_fn=<DivBackward0>)


In [241]:
# compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[4899.9229, 4348.5498, 2818.2456],
        [8353.9248, 8922.8604, 5602.9746]])
tensor([55.4761, 99.5447])


In [242]:
with torch.no_grad():
    w-= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [243]:
print(w)
print(b)

tensor([[ 0.6964,  0.6600,  0.1332],
        [-0.0716,  0.8812,  1.9049]], requires_grad=True)
tensor([-0.2328,  0.7989], requires_grad=True)


In [244]:
preds = model(inputs)
loss = mse(preds,targets)
print(loss)

tensor(4715.6377, grad_fn=<DivBackward0>)


# Train for multiple epochs

In [245]:
for i in range(100):
    preds = model(inputs)
    loss = mse(preds,targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad *1e-5
        w.grad.zero_()
        b.grad.zero_()

In [246]:
preds = model(inputs)
loss = mse(preds,targets)
print(loss)

tensor(131.8654, grad_fn=<DivBackward0>)


In [247]:
preds

tensor([[ 61.7571,  70.0171],
        [ 81.2266, 105.8608],
        [113.5395, 121.6254],
        [ 47.5905,  35.0893],
        [ 84.6985, 129.4482]], grad_fn=<AddBackward0>)

In [248]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# Now utilizing the pytorch builtin functions to make linear regression

In [249]:
import torch.nn as nn 

In [250]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)


In [251]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

# Dataset and DataLoarder

In [252]:
from torch.utils.data import TensorDataset

In [253]:
train_ds = TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [254]:
from torch.utils.data import DataLoader

In [255]:
batch_size = 5

train_d1 = DataLoader(train_ds,batch_size, shuffle=True)

In [256]:
for xb, yb in train_d1:
    print(xb)
    print(yb)
    break

tensor([[ 69.,  96.,  70.],
        [ 68.,  96.,  71.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [103.,  43.,  36.]])
tensor([[103., 119.],
        [104., 118.],
        [ 22.,  37.],
        [ 56.,  70.],
        [ 20.,  38.]])


In [257]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

# nn.Linear

In [258]:
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2216,  0.2022, -0.1897],
        [-0.2119, -0.4232, -0.2042]], requires_grad=True)
Parameter containing:
tensor([-0.5637, -0.0123], requires_grad=True)


In [259]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2216,  0.2022, -0.1897],
         [-0.2119, -0.4232, -0.2042]], requires_grad=True),
 Parameter containing:
 tensor([-0.5637, -0.0123], requires_grad=True)]

In [260]:
preds = model(inputs)
preds

tensor([[-11.3445, -52.6215],
        [-15.0688, -69.6128],
        [ -3.7412, -87.0087],
        [-21.4865, -47.3845],
        [ -9.7138, -69.5615],
        [-11.7683, -52.4102],
        [-15.4607, -69.3938],
        [ -4.1525, -87.4249],
        [-21.0626, -47.5958],
        [ -9.6819, -69.5538],
        [-11.7364, -52.4025],
        [-15.4926, -69.4015],
        [ -3.3493, -87.2278],
        [-21.5184, -47.3922],
        [ -9.2900, -69.7728]], grad_fn=<AddmmBackward>)

# Loss Function

In [261]:
import torch.nn.functional as F

In [262]:
loss_fn = F.mse_loss

In [263]:
loss = loss_fn(model(inputs),targets)
print(loss)

tensor(17853.9922, grad_fn=<MseLossBackward>)


# Optimizer

In [264]:
opt = torch.optim.SGD(model.parameters(),lr = 1e-5)

# Training

In [267]:
def fit(num_epochs,model,loss_fn,opt,train_d1):
    '''
    Train the model
    '''
    # number of iterations that will be realized
    for epoch in range(num_epochs):
        # train with the batches
        for xb, yb in train_d1:
            # generate predictions
            pred = model(xb)
            
            # calculate the loss
            loss = loss_fn(pred,yb)
            
            # compute gradients
            loss.backward()

            # update parameters using gradients
            opt.step()

            # reset gradients to zero
            opt.zero_grad()
        if (epoch + 1) % 10 == 0:
            print('Epoch[{}/{}], Loss: {:.4f}'.format(epoch+1,num_epochs,loss.item()))


In [268]:
fit(100,model,loss_fn,opt,train_d1)

Epoch[10/100], Loss: 386.8701
Epoch[20/100], Loss: 110.8057
Epoch[30/100], Loss: 115.4734
Epoch[40/100], Loss: 56.8332
Epoch[50/100], Loss: 58.6408
Epoch[60/100], Loss: 24.9537
Epoch[70/100], Loss: 25.3642
Epoch[80/100], Loss: 32.1732
Epoch[90/100], Loss: 27.5994
Epoch[100/100], Loss: 14.5351


In [270]:
preds = model(inputs)
preds

tensor([[ 57.8276,  71.6057],
        [ 79.5747,  98.4601],
        [122.6800, 135.7434],
        [ 25.9079,  44.3725],
        [ 94.4816, 111.0094],
        [ 56.6237,  70.6228],
        [ 78.9167,  98.1039],
        [122.7259, 136.1385],
        [ 27.1118,  45.3554],
        [ 95.0275, 111.6362],
        [ 57.1696,  71.2496],
        [ 78.3707,  97.4772],
        [123.3380, 136.0995],
        [ 25.3619,  43.7458],
        [ 95.6855, 111.9923]], grad_fn=<AddmmBackward>)

In [271]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [272]:
model(torch.tensor([[75,63,44.]]))

tensor([[53.8079, 68.4167]], grad_fn=<AddmmBackward>)